Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

tf.__version__

'2.4.1'

In [2]:
# Load Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pathlib
dataset_url = '/content/drive/MyDrive/Colab Notebooks/flowers'
data_dir = pathlib.Path(dataset_url)
data_dir

from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator()

dataset = data_generator.flow_from_directory(data_dir, class_mode='binary', batch_size=64)

Found 8646 images belonging to 6 classes.


In [4]:
batch_size = 32
img_height = 180
img_width = 180
x, y = dataset.next()

In [5]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "/content/drive/MyDrive/Colab Notebooks/flowers",
  validation_split=0.4,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "/content/drive/MyDrive/Colab Notebooks/flowers",
  validation_split=0.4,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8646 files belonging to 6 classes.
Using 5188 files for training.
Found 8646 files belonging to 6 classes.
Using 3458 files for validation.


In [6]:
class_names = train_dataset.class_names

In [7]:
def process_and_standardize_images(image,img_height, img_width):
    img = image.map(lambda x, y: (tf.image.resize(x, size=(30, 30)), y))
    return img

train_dataset = process_and_standardize_images(train_dataset, 3, 3)
test_dataset = process_and_standardize_images(test_dataset, 3, 3)

In [8]:
from tensorflow.keras import models, layers

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 30, 3)),
    tf.keras.layers.Dense(60, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"], optimizer="adam")

history = model.fit(train_dataset, validation_data=test_dataset, batch_size=128, epochs=1)

163/163 [==============================] - 1673s 10s/step - loss: 42.6019 - accuracy: 0.3158 - val_loss: 8.2007 - val_accuracy: 0.3707


In [10]:
from PIL import Image
import requests
url = "http://static.everypixel.com/ep-libreshot/1121/0442/0579/66480/11210442057966480324-solitary_yellow_flower.jpg"
img = Image.open(requests.get(url, stream=True).raw)

img = tf.image.resize(np.array(img), size=(30, 30))
print(img.shape)

predictions = model.predict(test_dataset.take(1))
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

(30, 30, 3)
This image most likely belongs to rose with a 23.20 percent confidence.
